In [54]:
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.initializers import Constant
from keras.layers import TextVectorization
import numpy as np
import pickle
import json

In [55]:
X = np.load('data/X.npy', allow_pickle=True)
y = np.load('data/y.npy')
embedding_matrix = np.load('data/embedding_matrix.npy')
num_tokens = np.load('data/num_tokens.npy').item()
embedding_dim = np.load('data/embedding_dim.npy').item()
max_tokens = np.load('data/max_tokens.npy').item()
max_len = np.load('data/max_len.npy').item()

In [56]:
vectorizer = TextVectorization(max_tokens=max_tokens,
                               standardize=None,
                               ngrams=(1, 2),
                               output_sequence_length=max_len)
vectorizer.adapt(X)

2022-02-02 14:26:56.871920: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [57]:
y = y.reshape((-1,1))

In [58]:
print(X.shape)
print(y.shape)

(7613,)
(7613, 1)


In [59]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [60]:
np.save('data/x_train.npy', x_train)
np.save('data/y_train.npy', y_train)
np.save('data/x_test.npy', x_test)
np.save('data/y_test.npy', y_test)

In [61]:
sequences_input = Input(shape=(1), dtype='string')

vectorizer_sequence = vectorizer(sequences_input)

embedding_layer = Embedding(num_tokens,
                            embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            trainable=False)
embedded_sequences = embedding_layer(vectorizer_sequence)

x = Dense(100, activation="relu")(embedded_sequences)
x = Dropout(0.2)(x)
x = Dense(50, activation="relu")(embedded_sequences)
x = Dropout(0.2)(x)
x = Dense(20, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(5, activation="relu")(x)
x = Dropout(0.2)(x)

preds = Dense(units=1,
              kernel_initializer='uniform',
              activation='sigmoid')(x)
model = Model(sequences_input, preds)

model.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_6 (TextV  (None, 68)               0         
 ectorization)                                                   
                                                                 
 embedding_6 (Embedding)     (None, 68, 100)           5000200   
                                                                 
 dense_27 (Dense)            (None, 68, 50)            5050      
                                                                 
 dropout_21 (Dropout)        (None, 68, 50)            0         
                                                                 
 dense_28 (Dense)            (None, 68, 20)            1020      
                                                           

In [62]:
n_samples = x_train.shape[0]

history = model.fit(x_train,
                    y_train,
                    validation_split=0.2,
                    batch_size=int(n_samples*0.1),
                    epochs=700)

model.save('models/model3.tf')

with open('models/history3.json', 'w') as f:
    json.dump(history.history, f)

Epoch 1/700


2022-02-02 14:26:58.410960: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 1s 90ms/step - loss: 0.6930 - accuracy: 0.5275 - val_loss: 0.6924 - val_accuracy: 0.5778


2022-02-02 14:26:59.268505: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 2/700
9/9 [==============================] - 0s 49ms/step - loss: 0.6923 - accuracy: 0.5632 - val_loss: 0.6915 - val_accuracy: 0.5773
Epoch 3/700
9/9 [==============================] - 0s 48ms/step - loss: 0.6915 - accuracy: 0.5640 - val_loss: 0.6900 - val_accuracy: 0.5782
Epoch 4/700
9/9 [==============================] - 0s 51ms/step - loss: 0.6902 - accuracy: 0.5643 - val_loss: 0.6884 - val_accuracy: 0.5788
Epoch 5/700
9/9 [==============================] - 0s 50ms/step - loss: 0.6889 - accuracy: 0.5662 - val_loss: 0.6866 - val_accuracy: 0.5831
Epoch 6/700
9/9 [==============================] - 0s 48ms/step - loss: 0.6875 - accuracy: 0.5688 - val_loss: 0.6843 - val_accuracy: 0.5857
Epoch 7/700
9/9 [==============================] - 0s 48ms/step - loss: 0.6859 - accuracy: 0.5697 - val_loss: 0.6821 - val_accuracy: 0.5868
Epoch 8/700
9/9 [==============================] - 0s 48ms/step - loss: 0.6846 - accuracy: 0.5717 - val_loss: 0.6803 - val_accuracy: 0.5879
Epoch 9/700
9/9 [===

2022-02-02 14:31:52.118249: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Assets written to: models/model3.tf/assets
